In [1]:
import yaml
import pathlib
import time

import torch
import torch.nn

import mnn.vision.models.vision_transformer.encoder.config as mnn_encoder_config
import mnn.vision.config as mnn_config
from mnn.vision.models.vision_transformer.e2e import (
    MyVisionTransformer
)

## UTILITIES

In [2]:
def inference_test(image: torch.Tensor, model: torch.nn.Module):
    t0 = time.time()
    output = model(image)
    t1 = time.time()
    print("Time taken:", t1 - t0, "seconds")
    print("Model's output shape:", output.shape)
    traced_model = torch.jit.trace(model.forward, image, check_trace=True, strict=True)
    return traced_model

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def read_yaml_file(file_path: pathlib.Path) -> dict:
    with file_path.open(mode="r") as f:
        # Python 3.11 need Loader
        return yaml.load(f, Loader=yaml.FullLoader)

## CHOOSE EXPERIMENT

In [4]:
experiment_number = int(input("Choose experiment Number and press Enter:"))
experiment_name = f"experiment{experiment_number}"

## INITIALIZATION

In [5]:
""" CONFIGURATION """
n = 1

model_config_as_dict = read_yaml_file(pathlib.Path(experiment_name) / "model.yaml")
model_config = mnn_encoder_config.MyVisionTransformerConfiguration.from_dict(
    model_config_as_dict["MyVisionTransformerConfiguration"]
)
encoder_config = model_config.encoder_config

hyperparameters_config_as_dict = read_yaml_file(pathlib.Path(experiment_name) / "hyperparameters.yaml")
hyperparameters_config = mnn_config.HyperparametersConfiguration.from_dict(hyperparameters_config_as_dict)
""" MODEL """

image_size = hyperparameters_config.image_size
# image to pytorch tensor
sequence_length = image_size.height
embedding_size = image_size.width
hidden_dim = embedding_size
image_RGB = torch.rand(n, 3, sequence_length, image_size.width) * 255

In [10]:
d_type = torch.float16
my_transformer = MyVisionTransformer(encoder_config, image_size, n_high_level_layers=model_config.n_high_level_layers, is_input_normalized=True, dtype=d_type)
image_RGB = image_RGB.to(dtype=d_type)
my_transformer.set_batch_size(n)


In [11]:
print(image_RGB.dtype)

torch.float32


In [12]:
for name, param in my_transformer.named_parameters():
    if param.dtype != d_type:
        print(name, param.dtype)

### Visualize the model

In [13]:
from torch.utils.tensorboard import SummaryWriter

# Create a SummaryWriter and log the graph
writer = SummaryWriter()
writer.add_graph(my_transformer, image_RGB)
writer.close()

/Users/emlvasilopoulos/Documents/projects/ml_interview_ready_code/.venv/lib/python3.11/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 12415 / 3145728 (0.4%)
Greatest absolute difference: 0.0003864765167236328 at index (0, 2, 846, 89) (up to 1e-05 allowed)
Greatest relative difference: 0.0015188026876884256 at index (0, 2, 236, 759) (up to 1e-05 allowed)
  _check_trace(


TracingCheckError: Tracing failed sanity checks!
ERROR: Graphs differed across invocations!
	Graph diff:
		  graph(%self.1 : __torch__.mnn.vision.models.vision_transformer.e2e.MyVisionTransformer,
		        %x.1 : Tensor):
		    %encoder_combinator_list : __torch__.torch.nn.modules.container.ModuleList = prim::GetAttr[name="encoder_combinator_list"](%self.1)
		    %_1 : __torch__.mnn.vision.models.vision_transformer.e2e.EncoderCombinator = prim::GetAttr[name="1"](%encoder_combinator_list)
		    %sigmoid : __torch__.torch.nn.modules.activation.Sigmoid = prim::GetAttr[name="sigmoid"](%self.1)
		    %encoder_combinator_list.1 : __torch__.torch.nn.modules.container.ModuleList = prim::GetAttr[name="encoder_combinator_list"](%self.1)
		    %_0.11 : __torch__.mnn.vision.models.vision_transformer.e2e.EncoderCombinator = prim::GetAttr[name="0"](%encoder_combinator_list.1)
		    %positional_encoder : __torch__.mnn.vision.models.vision_transformer.positional_encoders.sinusoidal.MyVisionPositional

In [10]:
traced_model = inference_test(image_RGB, my_transformer)
print("Number of parameters:", count_parameters(my_transformer))

RuntimeError: mat1 and mat2 must have the same dtype, but got Float and Half

In [14]:
import mnn.vision.models.vision_transformer.encoder.utils as mnn_encoder_utils
d_type = torch.float16
transformer_encoder = mnn_encoder_utils.get_transformer_encoder_from_config(model_config.encoder_config)
transformer_encoder.to(dtype=d_type)
src = torch.rand(10, 100, 1024).to(dtype=d_type)
out = transformer_encoder(src)
print(out.shape)
print(out.dtype)

torch.Size([10, 100, 1024])
torch.float16
